This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

In [ ]:
# reinstall tensorflow v 2.7.0 w. CUDA 11.2, cuCNN 8.1.0  
# compatibility issues. 
# uninstall tensorflow v 2.4, uninstall Cuda toolkit.  Keep Nvidia drivers. 
# num GPU is 1.  
# base conda env has tensorflow, no pytorch

In [1]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [2]:
# verify GPU, should be 1.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# Can ignore NUMA numa_node warning.

Num GPUs Available:  1


2022-06-21 19:46:06.673430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-21 19:46:06.679332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-21 19:46:06.679597: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Generative deep learning

## Text generation

### A brief history of generative deep learning for sequence generation

### How do you generate sequence data?

### The importance of the sampling strategy

**Reweighting a probability distribution to a different temperature**

In [0]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

### Implementing text generation with Keras

#### Preparing the data

**Downloading and uncompressing the IMDB movie reviews dataset**

In [0]:
#!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -xf aclImdb_v1.tar.gz

In [3]:
!ls ./aclImdb
# data already downloaded and extracted to ./aclImdb

/bin/bash: /home/jyoon/conda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
README	imdb.vocab  imdbEr.txt	test  train


**Creating a dataset from text files (one file = one sample)**

In [4]:
#import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 100006 files belonging to 1 classes.


2022-06-21 19:49:21.208748: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-21 19:49:21.210304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-21 19:49:21.210748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-21 19:49:21.211103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

**Preparing a `TextVectorization` layer**

In [5]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

**Setting up a language modeling dataset**

In [6]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

#### A Transformer-based sequence-to-sequence model

In [7]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**A simple Transformer-based language model**

In [8]:
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

### A text-generation callback with variable-temperature sampling

**The text-generation callback**

In [9]:
import numpy as np

tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

**Fitting the language model**

In [10]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200


2022-06-21 19:51:57.854603: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-06-21 19:51:58.458908: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100


391/391 [==============================] - ETA: 0s - loss: 5.3516== Generating with temperature 0.2
This movie movie had actually a hopes good having disappointed been because given i the remember acting that was the nonexistent intent at and the a video little rental and store i saw didnt bad take seems the to them see electronic it please it steal and from the all tv you
== Generating with temperature 0.5
This movie tv was version i as was well pretty as disappointed a me promising up with with bringing the time story the was [UNK] flawless is work a quite great out martial of insider the itself early and on was as not b to it and is many so of beautiful
== Generating with temperature 0.7
This movie man has has its got familiar to to have be interesting comparing and fantastic to personalities acquire are the frat country world and or world at that any this means movie that the womans crap control that but i i only also in have my been kid described there as
== Generating with temper

Epoch 6/200
391/391 [==============================] - ETA: 0s - loss: 4.3103== Generating with temperature 0.2
This movie movie is where a [UNK] guy takes is it forced not me only anymore breath anna south but park her former best [UNK] friend bob [UNK] [UNK] deserves [UNK] obviously less has of a [UNK] household gary of [UNK] sue already who drowns has and been some madly escaped in
== Generating with temperature 0.5
This movie movie has has the no gotten story so yeah far that in it time alaska my guy wife talking teds to life a story home like thats a fairly joke simple i soon can found say in i the found movie out to about be getting something boring i but
== Generating with temperature 0.7
This movie is is well not calculated for to the beat top scene beautifully of shot the by jaw lightning goldblum born is killers a sounds solid like draw his go lines on with for a the [UNK] [UNK] instantly of the the man bald and outlaw [UNK] connelly [UNK] who played
== Generating with temper

Epoch 11/200
391/391 [==============================] - ETA: 0s - loss: 4.1148== Generating with temperature 0.2
This movie is is so simply haunting perfect that piece it of is film to without grade any it doubt is the just most plain gripping awful acted cut film throat is meaning unbalanced and without it any the spoilers story not simply only it happens falls there way is but obviously
== Generating with temperature 0.5
This movie movie stinks is it damn ranks cliché among mainly the the worst most ever overrated made and of absolutely bad dire dialogue movie and scripts no made script my of fiancée choice has bad better bad fights acting however hollow please production someone student who should thinks comment this on
== Generating with temperature 0.7
This movie sucked was really worth about the horror movie business stalking except teenagers then terrorize pulls the it mask off incredible the oneliners woods are is very appalling brutal that but their obviously quick awkward dem

...

This movie movie was as by i far had the read [UNK] all the about disappointment the that movie helped was to complete say the how movie bad is and they boring say they that are the a directors nice other guy than is two a [UNK] [UNK] movie to [UNK] express
== Generating with temperature 1.5
This movie is is about a a dog drug but that the wont writer change even the for low [UNK] stretch to anyone half for this salma and hayek hour is in [UNK] it because might it possibly is be practically avoid unwatchable if to you turn then up take that that
391/391 [==============================] - 86s 221ms/step - loss: 3.6165
Epoch 177/200
391/391 [==============================] - ETA: 0s - loss: 3.6157== Generating with temperature 0.2
This movie movie is is not in bad serbian good [UNK] setting [UNK] yes [UNK] and chicks [UNK] nothing dancing and around the sight screen of are a relatively joke new the idea only was shocks ever me executed to and say sheridan the is [UNK] a [UNK] bell [UNK] 

This movie movie had has wonderful nothing music at to all live involved it jimmy has [UNK] never one makes thought it died lightly it has should no be hollywood enjoyed the not sets that still all havent the got ideas got and the to implied be donner black act the together
== Generating with temperature 1.0
This movie film was is [UNK] a to trace watch within especially an when age home sign movies of say brazil john this candy was conveys a both group person of and friends when over they two begin children to [UNK] see they what are has eating become [UNK] of there tempting
== Generating with temperature 1.5
This movie production sucks helps the multiple quality careers its in one the place insider a is very a talented powerful action documentary film about with how amazing situations that [UNK] shall leaders not michael deal moores with [UNK] the behind best it did was not in had the this liberal idea
391/391 [==============================] - 87s 221ms/step - loss: 3.6132
Epoch 182

This movie is has superb so piano much and as has commentary arrived on at this ireland bit film of was the it slow stated moving in and incomplete beautiful not french only to successful [UNK] especially it also has has a horror car theme nut [UNK] who which is shown the
== Generating with temperature 0.7
This movie [UNK] barely ahead made i by gave the it good as wasnt fast and and didnt never feel out any of of the the scenes characters which were i so would much have rather and been this running hard over to and enjoy we it didnt had know the better
== Generating with temperature 1.0
This movie movie is like one a part dark of reality a there serious are [UNK] several here kind probably of made a this movie movie is he good possessed in his front role for he every played thing i straight do than wasnt any that good a either bad this [UNK]
== Generating with temperature 1.5
This movie film really was deserves made a a good remarkably sales bad made situations by are everyone not who

This movie would was be love little to documentary recommend i it am is no very ordinary critical movie votes but or its cocaine a laurel must and fathom hardy that not kids only of saw know parks if commercials they who knew are was supposed excrement to that get organization posted
== Generating with temperature 0.7
This movie is is even [UNK] funny  and the actors knew how to create fools in themselves and can thats so hard to leave the film interesting simply because it is a start im sure they should use boxing in a budget of jack nicholson and filmed him over the cigarette
== Generating with temperature 1.0
This movie is truly the bad best acting movie little i action caught only it gary once cooper more set before the this big is camera just is scores better and to i boot guess it this follows movie is as a complete cinderella [UNK] almost my copying heroine works cry at
== Generating with temperature 1.5
This movie has was none massive of chuckles that but is it not really to wo

391/391 [==============================] - ETA: 0s - loss: 3.6046== Generating with temperature 0.2
This movie will is be a praised serious piece expert of from narration [UNK] as has well been as discussed an as aspect it goes may further suggest justice otherwise in someone the elses world attack probably every familiar one to who draw happens reasons with of each strong of sean the
== Generating with temperature 0.5
This movie is could extremely little likely well be made summed badly up theres space a up monster badly version place of on the top [UNK] of from hair the and guys buildings room [UNK] epidemic bad levels identical of to basement pot its milk gonna english teach a the lesson plot
== Generating with temperature 0.7
This movie is was a just total plain nonsense window study the awfully trip familiar through but a [UNK] stunning house french director cannibal don walking [UNK] sf seems filmmakers like to a want different to road your to average catch entertainment on expos

### Wrapping up